In [13]:
import os
try:
    for i in range(13):
        os.makedirs('D:\junggeyon\MR_aligned_png/Patient%d' %(i+1))
except:
    pass

In [14]:
import os
import numpy as np
import pydicom
from numpngw import write_png
###for 16bit image
basepath = 'D:\junggeyon\chan_find_pancreatobiliary/'
dir_list = os.listdir(basepath)

for patient in dir_list:
    if '001' in patient:
        sub_dir_list = os.listdir(basepath+patient)
        for mr in sub_dir_list:
            if 'MR' in mr and 'Align' in mr:
                MR_folder_path = basepath + patient + '/' + mr


def PNGandNPYsave(MR_voxel):
    np.save('D:\junggeyon\MR_aligned_png\Patient1\patient1MRvoxel', MR_voxel)
    for k in range(len(MR_voxel)):
        write_png('D:\junggeyon\MR_aligned_png\Patient1\pa1MR%d.png' %(k+1), MR_voxel[k])

In [15]:
def GetMRinfo(MR_folder_path):
    MR_dcm_files = []
    fileidx = []
    for top, dir, file in os.walk(MR_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            temp_ct_slice = pydicom.dcmread(MR_folder_path + '/' + filename)
            MR_dcm_files.append(temp_ct_slice)
            idxnum = temp_ct_slice.InstanceNumber
            fileidx.append(idxnum)
        break

    fileidx = np.array(fileidx)
    fileorder = np.argsort(fileidx)

    sorted_MR_file = []
    
    for i in range(len(fileidx)):
        sorted_MR_file.append(MR_dcm_files[fileorder[i]])
        
    return sorted_MR_file, len(fileidx)
    
sorted_MR_file, zsize = GetMRinfo(MR_folder_path)

In [16]:
def GetSlopeandIntercept(mr_dicomdata):
    slope = float(mr_dicomdata.RealWorldValueMappingSequence[0].RealWorldValueSlope)
    intercept = float(mr_dicomdata.RealWorldValueMappingSequence[0].RealWorldValueIntercept)
    return slope, intercept

In [17]:
def GraytoRGB(arr_slice):
    pixel_rgb = np.stack((arr_slice, )*3, axis = -1)
    return pixel_rgb

In [19]:
def MakeMRvoxel(sorted_MR_file, zsize):
    width = sorted_MR_file[0].Columns
    height = sorted_MR_file[0].Rows
    MR_voxel = np.zeros((zsize, height, width, 3), 'int32')

    for i in range(len(sorted_MR_file)):
        mr_dicomdata = sorted_MR_file[i]
        slope, intercept = GetSlopeandIntercept(mr_dicomdata)
        temp_arr_slice = slope * mr_dicomdata.pixel_array + intercept
        MR_voxel[i] = GraytoRGB(temp_arr_slice)

    return MR_voxel

MR_voxel = MakeMRvoxel(sorted_MR_file, zsize)


Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 200
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.16.840.1.114362.1.12105090.25050081628.616004206.676.139
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 2.16.840.1.114362.1
(0002, 0013) Implementation Version Name         SH: 'MIM 7.1.7'
(0002, 0016) Source Application Entity Title     AE: 'MIM'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', 'AXIAL']
(0008, 0012) Instance Creation Date              DA: '20220719'
(0008, 0013) Instance Creation Time              TM: '181125.458'
(0008, 0016) SOP Class UID             

In [20]:
def RefineMRvoxel(MR_voxel):
    print(np.min(MR_voxel))
    print(np.max(MR_voxel))
    MR_voxel = MR_voxel.astype('uint16')

    return MR_voxel

MR_voxel = RefineMRvoxel(MR_voxel)


0
39588


In [21]:
PNGandNPYsave(MR_voxel)
